In [1]:
import shutil

import geopandas as gpd
import leafmap

from geospatial_tools import DATA_DIR, TESTS_DIR
from geospatial_tools.planetary_computer.sentinel_2 import BestProductsForFeatures, download_and_process_sentinel2_asset
from geospatial_tools.raster import clip_raster_with_polygon
from geospatial_tools.stac import Asset
from geospatial_tools.vector import create_vector_grid_parallel, select_polygons_by_location, to_geopackage

In [2]:
TEST_TMP_DIR = TESTS_DIR / "test_notebooks/tmp_sentinel2"
TEST_TMP_DIR.mkdir(exist_ok=True)

### Initial pre-processing

The above layers were processed using QGIS.

For the purpose of this analysis, only the contiguous lower 48 states have been conserved; smaller islands/land masses 
have also been striped.

The S2 tiling grid has been trimmed to keep only the grid cells that overlap with the 
contiguous states.

Since our area of study is quite large, the `EPSG:5070` projection was chosen, as it
covers the whole area, introduces minimal distortion while preserving area.

The files below have also been saved in this repository.

In [3]:
USA_POLYGON_FILE = DATA_DIR / "usa_polygon_5070.gpkg"
S2_USA_GRID_FILE = DATA_DIR / "s2_grid_usa_polygon_5070.gpkg"
usa_polygon = gpd.read_file(USA_POLYGON_FILE)
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

In [4]:
usa_polygon

,AFFGEOID,GEOID,NAME,geometry
0,0100000US,US,United States,"MULTIPOLYGON (((-2116048.733 3142966.552, -211..."


In [5]:
s2_grid

,name,folders,description,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1386334.944 2487548.77 0, -..."
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-976300.478 2523767.452 0, -..."
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-960099.705 2622374.255 0, -..."
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1008622.024 2325748.358 0, ..."
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-992478.385 2424861.34 0, -8..."
...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1515431.586 2304192.826 0, ..."
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1448525.813 2089886.667 0, ..."
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1371006.917 2586590.133 0, ..."
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1355793.563 2685354.08 0, -..."


## Creating our grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

For the purpose of this notebook, the grid that will be created will use 10km by 10km squares to speed up 
processing.

In [6]:
grid_size = 10000
bbox = usa_polygon.total_bounds
grid_10km_filename = TEST_TMP_DIR / "polygon_grid_10km.gpkg"
print("Starting processing for [create_vector_grid_parallel]")
grid_10km = create_vector_grid_parallel(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
print(f"Printing len(grid_parallel) to check if grid contains same amount of polygons : {len(grid_10km)}")
to_geopackage(gdf=grid_10km, filename=grid_10km_filename)

Starting processing for [create_vector_grid_parallel]
[2025-05-22 14:12:05] INFO       [MainThread][geospatial_tools.vector] Creating grid coordinates for bounding box [[-2356113.74289801   301469.31619713  2258154.44089948  3165721.6501298 ]]
[2025-05-22 14:12:05] INFO       [MainThread][geospatial_tools.vector] Creating flattened grid coordinates
[2025-05-22 14:12:05] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2025-05-22 14:12:05] INFO       [MainThread][geospatial_tools.vector] Allocating polygon array for [132594] polygons
[2025-05-22 14:12:05] INFO       [MainThread][geospatial_tools.vector] Creating polygons from chunks
[2025-05-22 14:12:07] INFO       [MainThread][geospatial_tools.vector] Managing properties
[2025-05-22 14:12:07] INFO       [MainThread][geospatial_tools.utils] Creating EPSG code from following input : [EPSG:5070]
[2025-05-22 14:12:07] INFO       [MainThread][geospatial_tools.vector] Creating spatial index
[2025-05-22 14:12:07] I

PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/polygon_grid_10km.gpkg')

### Selecting the useful polygons

Now, since our grid was created using the extent of our input polygon (continental USA), we need to filter out the polygons that do not intersect with it.

Doing this in Python is not the most efficient way to do things, but since it's a step that shouldn't be done over and over, it's not that critical.

If ever you need to do this step in an efficient way because the data is just too big or too complex, it would be better off going through QGIS, PyGQIS, GDAL or 
some other more efficient way to do this operation.

In [7]:
intersecting_polygons_filename = TEST_TMP_DIR / "intersecting_polygon_grid_10km.gpkg"
print("Starting intersect selection")
intersecting_polygons = select_polygons_by_location(grid_10km, usa_polygon, num_of_workers=4)

# Optionally save to a new file
to_geopackage(intersecting_polygons, intersecting_polygons_filename)

Starting intersect selection
[2025-05-22 14:12:08] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 4
[2025-05-22 14:12:13] INFO       [MainThread][geospatial_tools.vector] Concatenating results
[2025-05-22 14:12:13] INFO       [MainThread][geospatial_tools.vector] Creating spatial index
[2025-05-22 14:12:13] INFO       [MainThread][geospatial_tools.vector] Filtering columns of the results
[2025-05-22 14:12:13] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2025-05-22 14:12:13] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/intersecting_polygon_grid_10km.gpkg] took 0.43907809257507324 seconds to write.


PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/intersecting_polygon_grid_10km.gpkg')

## Data processing pipeline prototype

### Finding the best image for each S2 tiling grid

In [8]:
# This is the full list of S2 grids
s2_tile_grid_list = s2_grid["name"].to_list()
s2_tile_grid_list

['12TUP',
 '12TYQ',
 '12TYR',
 '12TYN',
 '12TYP',
 '12TYS',
 '12TYT',
 '11SMB',
 '11SMC',
 '11SLV',
 '11SMA',
 '11SMS',
 '12UUV',
 '11SMT',
 '11SMD',
 '11SMR',
 '12UUU',
 '11SNA',
 '12TWS',
 '11SNB',
 '12TWT',
 '11SMU',
 '12TWQ',
 '11SMV',
 '12TWR',
 '12TXM',
 '11SNS',
 '12TXN',
 '11SNC',
 '12TXK',
 '11SND',
 '12TXL',
 '11SKD',
 '12TXR',
 '12TXS',
 '11SKB',
 '12TXP',
 '11SKC',
 '12TXQ',
 '11SKU',
 '12TYL',
 '11SKV',
 '12TYM',
 '12TXT',
 '11SKT',
 '12TYK',
 '11SLC',
 '11SLD',
 '11SLA',
 '11SLB',
 '11SLT',
 '11SLU',
 '19TBF',
 '19TBG',
 '19TDM',
 '19TDN',
 '19TEJ',
 '19TEK',
 '19TEN',
 '19TEL',
 '19TEM',
 '18STE',
 '18STF',
 '19TCG',
 '18STC',
 '19TCH',
 '18STD',
 '18STJ',
 '19TCF',
 '19TCL',
 '18STG',
 '19TCM',
 '18STH',
 '19TCJ',
 '19TCK',
 '19TDF',
 '19TDG',
 '19TDK',
 '19TDL',
 '19TDJ',
 '16TCQ',
 '16TCR',
 '16TCN',
 '16TCP',
 '16TDK',
 '16TDL',
 '16TCS',
 '16TCT',
 '16TDP',
 '15RWQ',
 '16TDQ',
 '16TDM',
 '15RXN',
 '16TDN',
 '15RXP',
 '16TDT',
 '16TEK',
 '16TDR',
 '16TDS',
 '16SGB',


In [9]:
# The list is a bit long, so we'll be continuing this notebook with the following subset
s2_tile_grid_subset_list = ["10TDK", "10TEK", "10SDJ"]

### Building our processing list

First, let's make create a subselection of our dataset

In [10]:
S2_USA_GRID_FILE = DATA_DIR / "s2_grid_usa_polygon_5070.gpkg"
s2_grid = gpd.read_file(S2_USA_GRID_FILE)
s2_grid

,name,folders,description,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1386334.944 2487548.77 0, -..."
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-976300.478 2523767.452 0, -..."
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-960099.705 2622374.255 0, -..."
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1008622.024 2325748.358 0, ..."
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-992478.385 2424861.34 0, -8..."
...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1515431.586 2304192.826 0, ..."
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1448525.813 2089886.667 0, ..."
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1371006.917 2586590.133 0, ..."
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-1355793.563 2685354.08 0, -..."


In [11]:
# Creating our S2 grid tile subset

S2_USA_GRID_SUBSET_FILE = TEST_TMP_DIR / "s2_grid_usa_polygon_5070_subset.gpkg"
s2_grid_subset = s2_grid[s2_grid["name"].isin(s2_tile_grid_subset_list)]

# Optionally save to geopackage
to_geopackage(gdf=s2_grid_subset, filename=S2_USA_GRID_SUBSET_FILE)

[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/s2_grid_usa_polygon_5070_subset.gpkg] took 0.02459549903869629 seconds to write.


PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/s2_grid_usa_polygon_5070_subset.gpkg')

In [12]:
s2_grid_subset

,name,folders,description,geometry
841,10SDJ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-2357236.175 2210256.163 0, ..."
846,10TDK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-2329019.777 2307113.875 0, ..."
857,10TEK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,"MULTIPOLYGON Z (((-2233778.019 2279366.081 0, ..."


In [13]:
# Creating our polygon grid subset

intersect_polygons_subset_filename = TEST_TMP_DIR / "intersecting_polygon_grid_10km_subset.gpkg"
print("Starting intersect selection")
intersect_polygons_subset = select_polygons_by_location(
    intersecting_polygons, s2_grid_subset, predicate="within", num_of_workers=4
)
# intersect_polygons_subset = gpd.read_file(intersect_polygons_subset_filename)

# Optionally save to a new file
to_geopackage(intersect_polygons_subset, intersect_polygons_subset_filename)

Starting intersect selection
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 4
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Concatenating results
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Creating spatial index
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Dropping duplicates
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Filtering columns of the results
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2025-05-22 14:12:14] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/intersecting_polygon_grid_10km_subset.gpkg] took 0.03481006622314453 seconds to write.


PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/intersecting_polygon_grid_10km_subset.gpkg')

### Finding the best products for our subset use case

In [14]:
s2_feature_name_columns = "name"
vector_column_name = "s2_tiles"

# Initiating our client
best_products_client = BestProductsForFeatures(
    sentinel2_tiling_grid=s2_grid_subset,
    sentinel2_tiling_grid_column=s2_feature_name_columns,
    vector_features=intersect_polygons_subset,
    vector_features_column=vector_column_name,
    max_cloud_cover=15,
)

In [15]:
# NBVAL_IGNORE_OUTPUT

# Executing the search
#
# This search look only for complete products, meaning products with less than
# 5 percent of nodata.

start_year = 2024
end_year = 2024
start_month = 6
end_month = 7

best_products_client.create_date_ranges(start_year, end_year, start_month, end_month)
products = best_products_client.find_best_complete_products()

[2025-05-22 14:12:15] INFO       [ThreadPoolExecutor-0_0][geospatial_tools.stac] Running STAC API search for the following parameters: 
	Date ranges : ['2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : {'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10SDJ']}}
[2025-05-22 14:12:15] INFO       [ThreadPoolExecutor-0_1][geospatial_tools.stac] Running STAC API search for the following parameters: 
	Date ranges : ['2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : {'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10TDK']}}
[2025-05-22 14:12:15] INFO       [ThreadPoolExecutor-0_2][geospatial_tools.stac] Running STAC API search for the following parameters: 
	Date ranges : ['2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : {'eo:cloud_cover': {'lt': 15}, 's2:mgrs_tile': {'in': ['10TEK']}}
[2025-05-22 14:12:16] WARNING    [MainThread][geospatial_tools.planetary_computer.sentinel_2] Warning, some of the input Sentinel 2 tiles do not have products covering the entire ti

In [16]:
products

{'10SDJ': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346',
  'cloud_cover': 0.006778,
  'no_data': 1.7e-05},
 '10TDK': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412',
  'cloud_cover': 0.000707,
  'no_data': 1e-05}}

In [17]:
# Selecting the best products for each vector tile
# This step is necessary as some of our vector polygons can be withing multiple S2 tiles.
# The best available S2 tile is therefore selected for each vector polygon.

best_results_path = TEST_TMP_DIR / "vector_tiles_with_s2tiles_subset.gpkg"
best_results = best_products_client.select_best_products_per_feature()
to_geopackage(best_results, best_results_path)

[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Creating temporary UUID field for join operations
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Starting process to find and identify contained features using spatial 'within' join operation
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Grouping results
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Cleaning and merging results
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Spatial join operation is completed
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.planetary_computer.sentinel_2] Writing best product IDs to dataframe
[2025-05-22 14:12:16] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2025-05-22 14:12:17] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/vector_tiles_with_s2tiles_sub

PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/vector_tiles_with_s2tiles_subset.gpkg')

### Visualizing the results


In [19]:
best_products_client.successful_results

{'10SDJ': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346',
  'cloud_cover': 0.006778,
  'no_data': 1.7e-05},
 '10TDK': {'id': 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412',
  'cloud_cover': 0.000707,
  'no_data': 1e-05}}

In [20]:
best_products_client.incomplete_results

['10TEK']

In [21]:
m_best_results = leafmap.Map(center=[39.7, -123], zoom=8)
m_best_results.add_gdf(s2_grid_subset, layer_name="s2_tiles", style={"color": "red"})
m_best_results.add_gdf(best_results, layer_name="vector_tiles_s2_grid", style={"color": "green"})
m_best_results

Map(center=[39.7, -123], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [22]:
# NBVAL_IGNORE_OUTPUT
group_by_product = best_results.groupby("best_s2_product_id")["feature_id"].agg(list).reset_index()

In [23]:
product_list = group_by_product["best_s2_product_id"].tolist()
sorted(product_list)

['S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346',
 'S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412']

### Downloading and processing Sentinel 2 products

#### Downloading and preparing Sentinel 2 products

In [24]:
# NBVAL_IGNORE_OUTPUT
product_asset_list = []
bands = ["B04", "B08"]
download_directory = TEST_TMP_DIR / "example_s2_download_and_processing"

for p in product_list:
    processed_product = download_and_process_sentinel2_asset(
        product_id=p, product_bands=bands, base_directory=download_directory, target_projection=5070
    )

    product_asset_list.append(processed_product)

[2025-05-22 14:12:17] INFO       [MainThread][geospatial_tools.stac] Initiating STAC API search
[2025-05-22 14:12:18] INFO       [MainThread][geospatial_tools.planetary_computer.sentinel_2] [<Item id=S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346>]
[2025-05-22 14:12:18] INFO       [MainThread][geospatial_tools.stac] Downloading [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346] ...
[2025-05-22 14:12:18] INFO       [MainThread][geospatial_tools.stac] Downloading B04 from https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/S/DJ/2024/07/05/S2A_MSIL2A_20240705T185921_N0510_R013_T10SDJ_20240706T050346.SAFE/GRANULE/L2A_T10SDJ_A047200_20240705T190418/IMG_DATA/R10m/T10SDJ_20240705T185921_B04_10m.tif?st=2025-05-21T18%3A12%3A16Z&se=2025-05-22T18%3A57%3A16Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-05-22T14%3A18%3A43Z&ske=2025-05-29T14%3A18%3A43Z&sks=b&skv=2024-05-04&sig=pjoW8ExBmQ6wXXL51%2B2r0zxn

In [25]:
for p in product_asset_list:
    print(f"Asset ID : [{p.asset_id}]")
    print(f"Reprojected ID path : \n[{p.reprojected_asset_path}]\n")

Asset ID : [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346]
Reprojected ID path : 
[/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_reprojected.tif]

Asset ID : [S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412]
Reprojected ID path : 
[/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/S2A_MSIL2A_20240705T185921_R013_T10TDK_20240706T050412_reprojected.tif]



In [26]:
# Here, we are creating a new Asset object simply for convenience, from the printed outputs above

product = Asset(
    asset_id="S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346",
    reprojected_asset=download_directory / "S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_reprojected.tif",
)

#### Creating a new geodataframe of all the vector polygons that are within our selected product

In [27]:
s2_product_id = product.asset_id
product_path = product.reprojected_asset_path
product_id_series = group_by_product[group_by_product["best_s2_product_id"] == s2_product_id]
# Since it's grouped by product id, there should always be only one row in the series
feature_ids = product_id_series["feature_id"].iloc[0]
vector_features = best_results[best_results["feature_id"].isin(feature_ids)]
vector_features_path = TEST_TMP_DIR / "vector_features.gpkg"
to_geopackage(vector_features, TEST_TMP_DIR / "vector_features.gpkg")

[2025-05-22 14:14:00] INFO       [MainThread][geospatial_tools.vector] Starting writing process
[2025-05-22 14:14:00] INFO       [MainThread][geospatial_tools.vector] File [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/vector_features.gpkg] took 0.009784221649169922 seconds to write.


PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/vector_features.gpkg')

#### Creating our Sentinel 2 "chips" by clipping main products with our vector polygon grid

In [28]:
clip_list = clip_raster_with_polygon(
    raster_image=product_path,
    polygon_layer=vector_features_path,
    base_output_filename=s2_product_id,
    output_dir=download_directory / "test_sentinel2_clip",
)

[2025-05-22 14:14:00] INFO       [MainThread][geospatial_tools.raster] Number of workers used: 16
[2025-05-22 14:14:00] INFO       [MainThread][geospatial_tools.raster] Output path : [/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/test_sentinel2_clip]
[2025-05-22 14:14:00] INFO       [MainThread][geospatial_tools.raster] Clipping raster image with 73 polygons
[2025-05-22 14:14:02] INFO       [MainThread][geospatial_tools.raster] Clipping process finished


In [29]:
sorted(clip_list)

[PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/test_sentinel2_clip/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_clipped_0.tif'),
 PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/test_sentinel2_clip/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_clipped_1.tif'),
 PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/test_sentinel2_clip/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_clipped_10.tif'),
 PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2_download_and_processing/test_sentinel2_clip/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_clipped_11.tif'),
 PosixPath('/home/francispelletier/projects/geospatial_tools/tests/test_notebooks/tmp_sentinel2/example_s2

In [30]:
shutil.rmtree(TEST_TMP_DIR)